In [1]:
import os
import zipfile
import pandas as pd
from collections import defaultdict
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
from newspaper import Article
from urllib.parse import urlparse
from tqdm import tqdm
import spacy
import glob
import os
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
from newspaper import Article
from urllib.parse import urlparse
from tqdm import tqdm
import spacy

c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
input_dir = "../data/gdelt_exports_cache_concat"
output_dir = "../data/gdelt_exports_cache_sentiment"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Load FinBERT model
MODEL_NAME = 'yiyanghkust/finbert-tone'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME)
finbert = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, truncation=True)

# Keywords or NASDAQ-100 company names (you can expand this list!)
nasdaq_keywords = [
    "AAPL", "Apple",
    "MSFT", "Microsoft",
    "AMZN", "Amazon.com", "Amazon",
    "NVDA", "NVIDIA Corporation", "Nvidia",
    "TSLA", "Tesla",
    "GOOGL", "GOOG", "Alphabet Inc.", "Google", "Alphabet",
    "META", "Meta Platforms", "Meta",
    "AVGO", "Broadcom Inc.", "Broadcom",
    "COST", "Costco Wholesale Corporation", "Costco",
    "PEP", "PepsiCo, Inc.", "Pepsi",
    "ADBE", "Adobe Inc.", "Adobe",
    "INTC", "Intel Corporation", "Intel",
    "NFLX", "Netflix, Inc.", "Netflix",
    "NDX", "NASDAQ",
    "BNB", "Airbnb, Inc.", "Airbnb",
    "ADI", "Analog Devices, Inc.", "Analog Devices",
    "ADP", "Automatic Data Processing", "Automatic Data Processing, Inc.",
    "ADSK", "Autodesk, Inc.", "Autodesk",
    "AEP", "American Electric Power Company, Inc.", "American Electric Power",
    "AMAT", "Applied Materials, Inc.", "Applied Materials",
    "AMD", "Advanced Micro Devices, Inc.", "AMD", "Advanced Micro Devices",
    "AMGN", "Amgen Inc.", "Amgen",
    "ANSS", "ANSYS, Inc.", "ANSYS",
    "APP", "Applovin Corporation",
    "ARM", "Arm Holdings plc", "Arm Holdings",
    "ASML", "ASML Holding N.V.", "ASML","ASML Holding",
    "AXON", "Axon Enterprise, Inc.", "Axon","Axon Enterprise",
    "AZN", "AstraZeneca PLC", "AstraZeneca",
    "BIIB", "Biogen Inc.", "Biogen",
    "BKNG", "Booking Holdings Inc.", "Booking Holdings",
    "BKR", "Baker Hughes Company", "Baker Hughes",
    "CCEP", "Coca-Cola Europacific Partners plc", "Coca-Cola Europacific Partners",
    "CDNS", "Cadence Design Systems, Inc.", "Cadence Design Systems",
    "CDW", "CDW Corporation",
    "CEG", "Constellation Energy Corporation",
    "CHTR", "Charter Communications, Inc.", "Charter Communications",
    "CMCSA", "Comcast Corporation",
    "CPRT", "Copart, Inc.", "Copart",
    "CRWD", "CrowdStrike Holdings, Inc.", "CrowdStrike",
    "CSCO", "Cisco Systems, Inc.", "Cisco",
    "CSGP", "CoStar Group, Inc.", "CoStar Group",
    "CSX", "CSX Corporation",
    "CTAS", "Cintas Corporation",
    "CTSH", "Cognizant Technology Solutions Corporation",
    "DASH", "DoorDash, Inc.", "DoorDash",
    "DDOG", "Datadog, Inc.", "Datadog",
    "DXCM", "DexCom, Inc.", "DexCom",
    "EA", "Electronic Arts Inc.", "Electronic Arts",
    "EXC", "Exelon Corporation",
    "FANG", "Diamondback Energy, Inc.", "Diamondback Energy",
    "FAST", "Fastenal Company", "Fastenal"
    "FTNT", "Fortinet, Inc.", "Fortinet",
    "GEHC", "GE HealthCare Technologies Inc.", "GE HealthCare",
    "GFS", "GlobalFoundries Inc.", "GlobalFoundries",
    "GILD", "Gilead Sciences, Inc.", "Gilead Sciences",
    "HON", "Honeywell International Inc.", "Honeywell",
    "IDXX", "IDEXX Laboratories, Inc.", "IDEXX Laboratories",
    "INTU", "Intuit Inc.", "Intuit",
    "ISRG", "Intuitive Surgical, Inc.", "Intuitive Surgical",
    "KDP", "Keurig Dr Pepper Inc.", "Keurig Dr Pepper",
    "KHC", "The Kraft Heinz Company", "The Kraft Heinz",
    "KLAC", "KLA Corporation",
    "LIN", "Linde plc",
    "LRCX", "Lam Research Corporation",
    "LULU", "lululemon athletica Inc.", "lululemon",
    "MAR", "Marriott International",
    "MCHP", "Microchip Technology Incorporated",
    "MDLZ", "Mondelez International, Inc.", "Mondelez",
    "MELI", "MercadoLibre, Inc.", "MercadoLibre",
    "MNST", "Monster Beverage Corporation", "Monster Beverage",
    "MRVL", "Marvell Technology, Inc.", "Marvell Technology",
    "MSTR", "MicroStrategy Incorporated",
    "MU", "Micron Technology, Inc.", "Micron",
    "NXPI", "NXP Semiconductors N.V.", "NXP Semiconductors",
    "ODFL", "Old Dominion Freight Line, Inc.", "Old Dominion Freight Line",
    "ON", "ON Semiconductor Corporation",
    "ORLY", "O'Reilly Automotive, Inc.", "O'Reilly Automotive",
    "PANW", "Palo Alto Networks, Inc.", "Palo Alto Networks",
    "PAYX", "Paychex, Inc.", "Paychex",
    "PCAR", "PACCAR Inc.", "PACCAR",
    "PDD", "PDD Holdings Inc.", "PDD Holdings",
    "PLTR", "Palantir Technologies Inc.", "Palantir Technologies",
    "PYPL", "PayPal Holdings, Inc.", "PayPal",
    "QCOM", "QUALCOMM Incorporated",
    "REGN", "Regeneron Pharmaceuticals, Inc.", "Regeneron Pharmaceuticals",
    "ROP", "Roper Technologies, Inc.", "Roper Technologies",
    "ROST", "Ross Stores, Inc.", "Ross Stores",
    "SBUX", "Starbucks Corporation", "Starbucks",
    "SHOP", "Shopify Inc.", "Shopify",
    "SNPS", "Synopsys, Inc.", "Synopsys",
    "TEAM", "Atlassian Corporation Plc", "Atlassian",
    "TMUS", "T-Mobile US, Inc.", "T-Mobile US",
    "TTD", "The Trade Desk, Inc.", "The Trade Desk",
    "TTWO", "Take-Two Interactive Software, Inc.", "Take-Two Interactive",
    "TXN", "Texas Instruments Incorporated", "Texas Instruments",
    "VRSK", "Verisk Analytics, Inc.", "Verisk Analytics",
    "VRTX", "Vertex Pharmaceuticals Incorporated", "Vertex Pharmaceuticals",
    "WBD", "Warner Bros. Discovery, Inc.", "Warner Bros. Discovery",
    "WDAY", "Workday, Inc.", "Workday",
    "XEL", "Xcel Energy Inc.", "Xcel Energy",
    "ZS", "Zscaler, Inc.", "Zscaler",
]

'''
# Finance-focused domains
trusted_financial_domains = [
    'reuters.com', 'bloomberg.com', 'cnbc.com', 'ft.com', 'wsj.com',
    'forbes.com', 'marketwatch.com', 'seekingalpha.com', 'investing.com'
]
'''

Device set to use cpu


In [ ]:

# Domain filter
'''
def is_financial_domain(url):
    try:
        domain = urlparse(url).netloc.lower()
        return any(fin_domain in domain for fin_domain in trusted_financial_domains)
    except:
        return False
'''
# Keyword match in title
def title_mentions_ndx(text, keywords):
    return any(k.lower() in text.lower() for k in keywords)

# Named Entity Recognition for ORG/PRODUCT
def extract_entities(text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents if ent.label_ in ("ORG", "PRODUCT")]

# Sentiment inference
def analyze_sentiment(text):
    try:
        result = finbert(text[:512])[0]
        return {'label': result['label'], 'score': result['score']}
    except:
        return {'label': 'NEUTRAL', 'score': 0.0}

# Main GDELT processor
def process_gdelt_file(gdelt_file, max_articles=200):
    df = pd.read_csv(gdelt_file, sep=',', header=None, low_memory=False)
    df.columns = [f"col_{i}" for i in range(len(df.columns))]

    # Target columns for filtering (GDELT "themes"/"categories" start around col_16 in V2)
    target_cols = ['col_16', 'col_17', 'col_18', 'col_19']
    keywords = {'BUS', 'BUSINESS', 'COMPANY'}

    # Filter rows containing business keywords in target columns
    df_filtered = df[df[target_cols].apply(lambda row: any(word in str(cell) for word in keywords for cell in row), axis=1)]

    if df_filtered.empty:
        print(f"⚠️ No relevant business-related rows found in {gdelt_file}")
        return pd.DataFrame()

    df_filtered['url'] = df_filtered[f"col_{len(df_filtered.columns) - 1}"]

    try:
        df_filtered['date'] = pd.to_datetime(df_filtered['col_1'], format='%Y%m%d', errors='coerce')
    except:
        df_filtered['date'] = pd.NaT

    print(f"✅ Loaded {len(df_filtered)} filtered rows from {gdelt_file}")

    sentiment_results = []
    seen_urls = set()

    for _, row in tqdm(df_filtered.iterrows(), total=min(len(df_filtered), max_articles * 2)):
        url = row['url']
        if url in seen_urls or not isinstance(url, str) or not url.startswith("http"):
            continue
        seen_urls.add(url)

        try:
            article = Article(url)
            article.download()
            article.parse()
            title = article.title or ""
        except:
            continue

        # Relevance check
        if title_mentions_ndx(title, nasdaq_keywords):
            relevant = True
        else:
            org_entities = extract_entities(title)
            relevant = any(k.lower() in ' '.join(org_entities).lower() for k in nasdaq_keywords)

        if not relevant:
            continue

        article_text = article.text.strip()
        if len(article_text) < 100:
            continue

        sentiment = analyze_sentiment(article_text)
        sentiment_results.append({
            'date': row['date'],
            'url': url,
            'label': sentiment['label'].upper().strip(),
            'score': sentiment['score'],
            'text': article_text[:200]
        })

        if len(sentiment_results) >= max_articles:
            break

    return pd.DataFrame(sentiment_results)


In [5]:
def aggregate_daily_sentiment(df_sentiments):
    label_map = {'POSITIVE': 1, 'NEGATIVE': -1, 'NEUTRAL': 0}
    df_sentiments['label_num'] = df_sentiments['label'].map(label_map)

    agg = df_sentiments.groupby('date').agg({
        'label_num': 'mean',
        'score': 'mean',
        'url': 'count'
    }).rename(columns={
        'label_num': 'avg_sentiment',
        'score': 'avg_score',
        'url': 'article_count'
    })

    return agg.reset_index()

In [6]:
from datetime import datetime, timedelta

In [ ]:
# 🔁 Loop over all *_combined.csv files
combined_files = sorted([
    f for f in os.listdir(input_dir) if f.endswith('_combined.csv')
])

for file in combined_files:
    file_path = os.path.join(input_dir, file)
    date_str = file[:8]  # Extract YYYYMMDD

    print(f"\n📅 Processing: {file}")

    sentiment_df = process_gdelt_file(file_path, max_articles=300)
    sentiment_out_path = os.path.join(output_dir, f"{date_str}_sentiments.csv")
    sentiment_df.to_csv(sentiment_out_path, index=False)

    daily_score_df = aggregate_daily_sentiment(sentiment_df)
    daily_score_path = os.path.join(output_dir, f"{date_str}_daily_score.csv")
    daily_score_df.to_csv(daily_score_path, index=False)


📅 Processing: 20250101_combined.csv


C:\Users\PC\AppData\Local\Temp\ipykernel_17480\1628000028.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['url'] = df_filtered[f"col_{len(df_filtered.columns) - 1}"]
C:\Users\PC\AppData\Local\Temp\ipykernel_17480\1628000028.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['date'] = pd.to_datetime(df_filtered['col_1'], format='%Y%m%d', errors='coerce')


✅ Loaded 641 filtered rows from ../data/gdelt_exports_cache_concat\20250101_combined.csv


 29%|██▊       | 172/600 [01:49<05:59,  1.19it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
641it [08:09,  1.31it/s]                         



📅 Processing: 20250102_combined.csv


...

1208it [14:51,  1.36it/s]                        



📅 Processing: 20250104_combined.csv


C:\Users\PC\AppData\Local\Temp\ipykernel_17480\1628000028.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['url'] = df_filtered[f"col_{len(df_filtered.columns) - 1}"]
C:\Users\PC\AppData\Local\Temp\ipykernel_17480\1628000028.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['date'] = pd.to_datetime(df_filtered['col_1'], format='%Y%m%d', errors='coerce')


✅ Loaded 619 filtered rows from ../data/gdelt_exports_cache_concat\20250104_combined.csv


 16%|█▌        | 97/600 [01:08<05:23,  1.55it/s]